In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.3.0 pyspark-shell'

import geohash
import sys
import time
import json
from pymongo import MongoClient
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import datetime as dt
def geoH(a,b):
    return geohash.encode(a, b, 3)

def sendDataToDB(iter):
    client = MongoClient()
    db=client.fit5148_assignment_db 
    climate_stream = db.climate_stream
    hotspot_stream = db.hotspot_stream
    Producer2_no=0
    Producer3_no=0
    Producer1 = []
    Producer2 = {}
    Producer3 = {}
    for record in iter:
        data = json.loads(record[1])
        if data.get("send_id") == 'producer1':
            climate_data={}
            climate_data["create_time"]=dt.datetime.strptime(data.get("create_time"),"%Y-%m-%dT%H:%M:%S.000Z")
            climate_data["geoH"]=geoH(data.get("latitude"),data.get("longitude"))
            climate_data["latitude"]=data.get("latitude")
            climate_data["longitude"]=data.get("longitude")
            climate_data["air_temperature_celcius"]=data.get("air_temperature_celcius")
            climate_data["relative_humidity"]=data.get("relative_humidity")
            climate_data["windspeed_knots"]=data.get("windspeed_knots")
            climate_data["max_wind_speed"]=data.get("max_wind_speed")
            climate_data["precipitation"]=data.get("precipitation")                
            Producer1.append(climate_data)
        if data.get("send_id") == 'producer2':                
            Producer2_no=Producer2_no+1
            Producer2["geoH"]=geoH(data.get("latitude"),data.get("longitude"))
            Producer2["latitude"]=data.get("latitude")
            Producer2["longitude"]=data.get("longitude")
            Producer2["confidence"]=data.get("confidence")
            Producer2["surface_temperature_celcius"]=data.get("surface_temperature_celcius")   
        if data.get("send_id") == 'producer3':    
            Producer3_no=Producer3_no+1
            Producer3["geoH"]=geoH(data.get("latitude"),data.get("longitude"))
            Producer3["latitude"]=data.get("latitude")
            Producer3["longitude"]=data.get("longitude")
            Producer3["confidence"]=data.get("confidence")
            Producer3["surface_temperature_celcius"]=data.get("surface_temperature_celcius")
    
    if Producer2_no==0 and Producer3_no==0:
        for i in  Producer1:
            try :
                climate_stream.insert_one(i)
            except Exception as ex:
                print("Exception Occured. Message: {0}".format(str(ex)))
        client.close()
    
    if Producer2_no==1 and Producer3_no==0:
        for i in Producer1:
            if Producer2["geoH"]==i["geoH"]:
                hotspot_data=Producer2
                hotspot_data["match_date"]=i["create_time"]
                try :
                    hotspot_stream.insert_one(hotspot_data)
                    climate_stream.insert_one(i)
                except Exception as ex:
                    print("Exception Occured. Message: {0}".format(str(ex)))
        client.close()
    
    if Producer3_no==1 and Producer2_no==0:
        for i in Producer1:
            if Producer3["geoH"]==i["geoH"]:
                hotspot_data=Producer3
                hotspot_data["match_date"]=i["create_time"]
                try:
                    hotspot_stream.insert_one(hotspot_data)
                    climate_stream.insert_one(i)
                except Exception as ex:
                    print("Exception Occured. Message: {0}".format(str(ex)))
        client.close() 

    if Producer2_no==1 and Producer3_no==1: 
        if Producer3["geoH"]==Producer2["geoH"]:
            Producer2["confidence"]=(Producer2["confidence"]+Producer3["confidence"])/2
            Producer2["surface_temperature_celcius"]=(Producer2["surface_temperature_celcius"]+Producer3["surface_temperature_celcius"])/2
            for i in Producer1:
                if Producer2["geoH"]==i["geoH"]:
                    hotspot_data=Producer2
                    hotspot_data["match_date"]=i["create_time"]
                    try :
                        hotspot_stream.insert_one(hotspot_data)
                        climate_stream.insert_one(i)
                    except Exception as ex:
                        print("Exception Occured. Message: {0}".format(str(ex)))            
        else:
            for i in Producer1:
                if Producer2["geoH"]==i["geoH"]:
                    hotspot_data=Producer2
                    hotspot_data["match_date"]=i["create_time"]
                    try :
                        hotspot_stream.insert_one(hotspot_data)
                        climate_stream.insert_one(i)
                    except Exception as ex:
                        print("Exception Occured. Message: {0}".format(str(ex)))
            for i in Producer1:
                if Producer3["geoH"]==i["geoH"]:
                    hotspot_data=Producer3
                    hotspot_data["match_date"]=i["create_time"]
                    try:
                        hotspot_stream.insert_one(hotspot_data)
                        climate_stream.insert_one(i)
                    except Exception as ex:
                        print("Exception Occured. Message: {0}".format(str(ex)))        
        client.close()
        
        
        
        
        
        
        
        
        #                     hotspot_stream.insert_one(Producer2)
#                 except Exception as ex:
#                     print("Exception Occured. Message: {0}".format(str(ex)))
#         if h_key in producer1_dic:
#             for item in producer1_dic[h_key]:
#                 Producer2["match_date"]=item["create_time"]
#                 try:
#                     climate_stream.insert_one(item)
#                     hotspot_stream.insert_one(Producer2)
#                 except Exception as ex:
#                     print("Exception Occured. Message: {0}".format(str(ex)))
#         client.close()
            
    
    
        
n_secs = 10
topic = "AssignmentB"

conf = SparkConf().setAppName("KafkaStreamProcessor").setMaster("local[2]")
sc = SparkContext.getOrCreate()
if sc is None:
    sc = SparkContext(conf=conf)
sc.setLogLevel("WARN")
ssc = StreamingContext(sc, n_secs)
    
kafkaStream = KafkaUtils.createDirectStream(ssc, [topic], {
                        'bootstrap.servers':'127.0.0.1:9092', 
                        'group.id':'week12-group', 
                        'fetch.message.max.bytes':'15728640',
                        'auto.offset.reset':'largest'})
                        # Group ID is completely arbitrary
pairs = kafkaStream.map(lambda word: (word, 1))
wordCounts = pairs.reduceByKey(lambda x, y: x + y)                           
wordCounts.pprint()
lines = kafkaStream.foreachRDD(lambda rdd: rdd.foreachPartition(sendDataToDB))

ssc.start()
time.sleep(7200) # Run stream for 10 minutes just in case no detection of producer
# ssc.awaitTermination()
ssc.stop(stopSparkContext=True,stopGraceFully=True)

-------------------------------------------
Time: 2019-05-23 00:25:10
-------------------------------------------
((None, '{"latitude": -37.227, "confidence": 54, "send_id": "producer3", "surface_temperature_celcius": 31, "longitude": 141.146, "create_time": "2019-05-23T00:25:08.000Z"}'), 1)
((None, '{"relative_humidity": "43.8", "longitude": 143.7149, "air_temperature_celcius": 12, "precipitation": " 0.16G", "send_id": "producer1", "windspeed_knots": "6.7", "max_wind_speed": "14", "create_time": "2019-05-23T00:25:05.000Z", "latitude": -36.3799}'), 1)

-------------------------------------------
Time: 2019-05-23 00:25:20
-------------------------------------------
((None, '{"relative_humidity": "40.8", "longitude": 144.386, "air_temperature_celcius": 11, "precipitation": " 0.24G", "send_id": "producer1", "windspeed_knots": "12.2", "max_wind_speed": "20", "create_time": "2019-05-23T00:25:10.000Z", "latitude": -37.296}'), 1)
((None, '{"relative_humidity": "56.7", "longitude": 142.7605, "

-------------------------------------------
Time: 2019-05-23 00:27:00
-------------------------------------------
((None, '{"relative_humidity": "37.9", "longitude": 141.5759, "air_temperature_celcius": 7, "precipitation": " 0.04G", "send_id": "producer1", "windspeed_knots": "5.7", "max_wind_speed": "9.9", "create_time": "2019-05-23T00:26:55.000Z", "latitude": -36.429}'), 1)
((None, '{"relative_humidity": "49.6", "longitude": 142.3405, "air_temperature_celcius": 23, "precipitation": " 0.00I", "send_id": "producer1", "windspeed_knots": "6.3", "max_wind_speed": "13", "create_time": "2019-05-23T00:26:50.000Z", "latitude": -36.1002}'), 1)

-------------------------------------------
Time: 2019-05-23 00:27:10
-------------------------------------------
((None, '{"latitude": -37.2512, "confidence": 86, "send_id": "producer3", "surface_temperature_celcius": 60, "longitude": 144.3799, "create_time": "2019-05-23T00:27:05.000Z"}'), 1)
((None, '{"create_time": "2019-05-23T00:27:08.000Z", "longitu

-------------------------------------------
Time: 2019-05-23 00:28:50
-------------------------------------------
((None, '{"relative_humidity": "50.7", "longitude": 143.4924, "air_temperature_celcius": 13, "precipitation": " 0.00I", "send_id": "producer1", "windspeed_knots": "6.1", "max_wind_speed": "13", "create_time": "2019-05-23T00:28:45.000Z", "latitude": -37.4437}'), 1)
((None, '{"relative_humidity": "56.3", "longitude": 146.155, "air_temperature_celcius": 22, "precipitation": " 0.08G", "send_id": "producer1", "windspeed_knots": "8", "max_wind_speed": "13", "create_time": "2019-05-23T00:28:40.000Z", "latitude": -36.273}'), 1)

-------------------------------------------
Time: 2019-05-23 00:29:00
-------------------------------------------
((None, '{"latitude": -36.8846, "confidence": 79, "send_id": "producer3", "surface_temperature_celcius": 52, "longitude": 145.1679, "create_time": "2019-05-23T00:28:52.000Z"}'), 1)
((None, '{"relative_humidity": "56.3", "longitude": 146.155, "ai

-------------------------------------------
Time: 2019-05-23 00:30:30
-------------------------------------------
((None, '{"relative_humidity": "48.2", "longitude": 142.6138, "air_temperature_celcius": 15, "precipitation": " 0.43G", "send_id": "producer1", "windspeed_knots": "14.7", "max_wind_speed": "25.1", "create_time": "2019-05-23T00:30:25.000Z", "latitude": -36.8264}'), 1)
((None, '{"relative_humidity": "50.7", "longitude": 142.986, "air_temperature_celcius": 15, "precipitation": " 0.02G", "send_id": "producer1", "windspeed_knots": "9.2", "max_wind_speed": "13", "create_time": "2019-05-23T00:30:20.000Z", "latitude": -38.038}'), 1)
((None, '{"latitude": -36.3879, "confidence": 82, "send_id": "producer3", "surface_temperature_celcius": 55, "longitude": 141.4277, "create_time": "2019-05-23T00:30:23.000Z"}'), 1)

-------------------------------------------
Time: 2019-05-23 00:30:40
-------------------------------------------
((None, '{"create_time": "2019-05-23T00:30:37.000Z", "longi

-------------------------------------------
Time: 2019-05-23 00:32:10
-------------------------------------------
((None, '{"relative_humidity": "43.7", "longitude": 147.167, "air_temperature_celcius": 10, "precipitation": " 0.00I", "send_id": "producer1", "windspeed_knots": "7.2", "max_wind_speed": "11.1", "create_time": "2019-05-23T00:32:05.000Z", "latitude": -38.226}'), 1)
((None, '{"relative_humidity": "50.3", "longitude": 143.6131, "air_temperature_celcius": 19, "precipitation": " 0.00I", "send_id": "producer1", "windspeed_knots": "7.7", "max_wind_speed": "11.1", "create_time": "2019-05-23T00:32:00.000Z", "latitude": -36.9194}'), 1)

-------------------------------------------
Time: 2019-05-23 00:32:20
-------------------------------------------
((None, '{"latitude": -36.6995, "confidence": 82, "send_id": "producer3", "surface_temperature_celcius": 56, "longitude": 142.9146, "create_time": "2019-05-23T00:32:10.000Z"}'), 1)
((None, '{"relative_humidity": "43.8", "longitude": 148.11

...

-------------------------------------------
Time: 2019-05-23 02:18:00
-------------------------------------------
((None, '{"relative_humidity": "50.7", "longitude": 148.109, "air_temperature_celcius": 14, "precipitation": " 0.12G", "send_id": "producer1", "windspeed_knots": "8.6", "max_wind_speed": "13", "create_time": "2019-05-23T02:17:54.000Z", "latitude": -37.461}'), 1)
((None, '{"latitude": -36.6085, "confidence": 80, "send_id": "producer3", "surface_temperature_celcius": 53, "longitude": 142.4451, "create_time": "2019-05-23T02:17:58.000Z"}'), 1)
((None, '{"relative_humidity": "38.4", "longitude": 142.6069, "air_temperature_celcius": 13, "precipitation": " 0.00G", "send_id": "producer1", "windspeed_knots": "11.5", "max_wind_speed": "16.9", "create_time": "2019-05-23T02:17:59.000Z", "latitude": -37.6836}'), 1)

-------------------------------------------
Time: 2019-05-23 02:18:10
-------------------------------------------
((None, '{"create_time": "2019-05-23T02:18:08.000Z", "longi

-------------------------------------------
Time: 2019-05-23 02:19:50
-------------------------------------------
((None, '{"relative_humidity": "53.6", "longitude": 149.341, "air_temperature_celcius": 18, "precipitation": " 0.00I", "send_id": "producer1", "windspeed_knots": "7.2", "max_wind_speed": "15", "create_time": "2019-05-23T02:19:49.000Z", "latitude": -37.382}'), 1)
((None, '{"relative_humidity": "39.4", "longitude": 148.102, "air_temperature_celcius": 10, "precipitation": " 0.00G", "send_id": "producer1", "windspeed_knots": "9.6", "max_wind_speed": "14", "create_time": "2019-05-23T02:19:44.000Z", "latitude": -37.446}'), 1)

-------------------------------------------
Time: 2019-05-23 02:20:00
-------------------------------------------
((None, '{"latitude": -36.4794, "confidence": 63, "send_id": "producer3", "surface_temperature_celcius": 41, "longitude": 144.5752, "create_time": "2019-05-23T02:19:58.000Z"}'), 1)
((None, '{"create_time": "2019-05-23T02:19:52.000Z", "longitude"

-------------------------------------------
Time: 2019-05-23 02:21:30
-------------------------------------------
((None, '{"relative_humidity": "30.8", "longitude": 142.785, "air_temperature_celcius": 6, "precipitation": " 0.00I", "send_id": "producer1", "windspeed_knots": "5.7", "max_wind_speed": "11.1", "create_time": "2019-05-23T02:21:24.000Z", "latitude": -36.275}'), 1)
((None, '{"relative_humidity": "52.5", "longitude": 149.336, "air_temperature_celcius": 17, "precipitation": " 0.00G", "send_id": "producer1", "windspeed_knots": "6.7", "max_wind_speed": "12", "create_time": "2019-05-23T02:21:29.000Z", "latitude": -37.585}'), 1)

-------------------------------------------
Time: 2019-05-23 02:21:40
-------------------------------------------
((None, '{"create_time": "2019-05-23T02:21:35.000Z", "longitude": 143.8854, "send_id": "producer2", "surface_temperature_celcius": 63, "confidence": 88, "latitude": -35.8164}'), 1)
((None, '{"relative_humidity": "55.5", "longitude": 149.307, "a

-------------------------------------------
Time: 2019-05-23 02:23:10
-------------------------------------------
((None, '{"relative_humidity": "49.6", "longitude": 142.3405, "air_temperature_celcius": 23, "precipitation": " 0.00I", "send_id": "producer1", "windspeed_knots": "6.3", "max_wind_speed": "13", "create_time": "2019-05-23T02:23:09.000Z", "latitude": -36.1002}'), 1)
((None, '{"relative_humidity": "62.2", "longitude": 143.803, "air_temperature_celcius": 20, "precipitation": " 0.00I", "send_id": "producer1", "windspeed_knots": "7.1", "max_wind_speed": "15", "create_time": "2019-05-23T02:23:04.000Z", "latitude": -38.16}'), 1)

-------------------------------------------
Time: 2019-05-23 02:23:20
-------------------------------------------
((None, '{"relative_humidity": "61.1", "longitude": 146.791, "air_temperature_celcius": 21, "precipitation": " 0.00I", "send_id": "producer1", "windspeed_knots": "6.6", "max_wind_speed": "11.1", "create_time": "2019-05-23T02:23:19.000Z", "latit

-------------------------------------------
Time: 2019-05-23 02:25:00
-------------------------------------------
((None, '{"relative_humidity": "41.7", "longitude": 143.4996, "air_temperature_celcius": 14, "precipitation": " 0.00G", "send_id": "producer1", "windspeed_knots": "12.3", "max_wind_speed": "18.1", "create_time": "2019-05-23T02:24:54.000Z", "latitude": -36.9364}'), 1)
((None, '{"create_time": "2019-05-23T02:24:57.000Z", "longitude": 143.1764, "send_id": "producer2", "surface_temperature_celcius": 47, "confidence": 74, "latitude": -37.7905}'), 1)
((None, '{"relative_humidity": "44.9", "longitude": 143.7313, "air_temperature_celcius": 17, "precipitation": " 0.24G", "send_id": "producer1", "windspeed_knots": "16.1", "max_wind_speed": "27", "create_time": "2019-05-23T02:24:59.000Z", "latitude": -36.3782}'), 1)
((None, '{"latitude": -36.4097, "confidence": 84, "send_id": "producer3", "surface_temperature_celcius": 58, "longitude": 145.8952, "create_time": "2019-05-23T02:24:52.000

In [1]:
import geohash
import sys
import time
import json
from pymongo import MongoClient
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

client = MongoClient()
db=client.fit5148_assignment_db
climate_stream = db.climate_stream
hotspot_stream = db.hotspot_stream
climate_stream.drop()
hotspot_stream.drop()